In this notebook I'll reconstruct the process and results of the DTCR algorithm presented in the "Learning Representations for Time Series Clustering".

In [1]:
# Imports
from torch.utils.data import DataLoader
from Utilities.DTCR import DTCRModel, DTCRConfig
from Utilities.UCRParser import read_dataset
from Utilities.DRNN import BidirectionalDRNN

In [2]:
train_ds, test_ds = read_dataset("Beef")
train_dl = DataLoader(train_ds, batch_size=2, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=2, shuffle=True)

config = DTCRConfig()
config.class_num = train_ds.number_of_labels
config.input_size = train_ds[0][0].shape[1]
config.num_steps = train_ds[0][0].shape[0]
config.batch_size = 2


Loading the Beef dataset...
The dataset Beef was loaded.


In [3]:
dtcr_model = DTCRModel(config)


In [4]:
encoder = dtcr_model.encoder
encoder


BidirectionalDRNN(
  (_regular_drnn): DRNN(
    (cells): Sequential(
      (0): GRU(1, 100)
      (1): GRU(100, 50)
      (2): GRU(50, 50)
    )
  )
  (_backwards_drnn): DRNN(
    (cells): Sequential(
      (0): GRU(1, 100)
      (1): GRU(100, 50)
      (2): GRU(50, 50)
    )
  )
)

In [5]:
for sample_data, sample_label in train_dl:
    out, hidden = encoder(sample_data)
    break


In [6]:
print("sample shape: {}".format(sample_data.shape))
print("out shape: {}".format(out.shape))
print("hidden length: {}".format(len(hidden)))
for hidden_layer in range(len(hidden)):
    print("hidden layer {} shape: {}".format(hidden_layer,hidden[hidden_layer].shape))

sample shape: torch.Size([2, 470, 1])
out shape: torch.Size([2, 470, 100])
hidden length: 6
hidden layer 0 shape: torch.Size([1, 2, 100])
hidden layer 1 shape: torch.Size([4, 2, 50])
hidden layer 2 shape: torch.Size([16, 2, 50])
hidden layer 3 shape: torch.Size([1, 2, 100])
hidden layer 4 shape: torch.Size([4, 2, 50])
hidden layer 5 shape: torch.Size([16, 2, 50])
